# Wind Turbine Blade Monitoring System

This notebook uses Pathway to process real-time wind turbine telemetry data from a NATS message queue, detect anomalies, and publish alerts back to NATS.

In [ ]:
import pathway as pw

## 1. Define Data Schema

First, we define the schema for the incoming telemetry data. This ensures that the data is structured correctly.

In [ ]:
class TelemetrySchema(pw.Schema):
   turbine_id: str
   timestamp: str
   blade_length: float
   blade_width: float
   vibration: int
   temperature: int

## 2. Ingest Data from NATS

We connect to the NATS server and read the telemetry data from the `turbine.telemetry` topic.

In [ ]:
telemetry_table = pw.io.nats.read(
   uri="nats://host.docker.internal:4222",
   topic="turbine.telemetry",
   format="json",
   schema=TelemetrySchema
)

## 3. Define Alerting Logic

A User-Defined Function (UDF) is created to check for conditions that should trigger an alert.

In [ ]:
@pw.udf
def detect_alerts(vibration, temperature):
   alerts = []
   if vibration > 80:
       alerts.append("High Vibration Detected")
   if temperature > 100:
       alerts.append("High Temperature Detected")
   return alerts

## 4. Process Data and Generate Alerts

The `detect_alerts` UDF is applied to the incoming data stream. We then filter out any entries that didn't generate an alert.

In [ ]:
alerts = telemetry_table.select(
   turbine_id=pw.this.turbine_id,
   timestamp=pw.this.timestamp,
   alert_type=detect_alerts(
       pw.this.vibration,
       pw.this.temperature
   )
)

# Filter rows with no alerts
alerts = alerts.flatten(pw.this.alert_type).filter(pw.this.alert_type.is_not_none())

## 5. Output Alerts to NATS

The generated alerts are published to the `turbine.alerts` topic on the NATS server.

In [ ]:
pw.io.nats.write(
   alerts.select(
       turbine_id=pw.this.turbine_id,
       timestamp=pw.this.timestamp,
       alert_type=pw.this.alert_type
   ),
   uri="nats://host.docker.internal:4222",
   topic="turbine.alerts",
   format="json"
)

## 6. Run the Pipeline

Finally, we start the Pathway pipeline to begin processing data.

In [ ]:
pw.run()